<a href="https://colab.research.google.com/github/seungjae6874/Deeplearning/blob/master/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn.init
import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'

#랜덤 시드 고정
torch.manual_seed(777)

#GPU 사용가능이면 랜덤 시드 고정
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

learning_rate = 0.001
training_epochs = 15 #전체 데이터를 15번 사용해서 학습시키겠다는 의미
batch_size = 100 # 한번 batch할때 학습시킬 데이터의 수를 의미, iteration은 epoch를 나누어서 실행하는 수이다
#ex epoch = 20이면 전체 데이터를 총 20번 반복해서 학습한다는 뜻이고 
#만약 iteration이 4이면 총 epoch를 20번 돌기위해 반복문을 4번 돌리는 것이고 한번 돌릴 때 즉 5번의 전체 데이터셋을 사용한다는 의미

#데이터로더를 사용하여 데이터를 다루기 위해 데이터셋 정의

mnist_train = dsets.MNIST(root='MNIST_data/', #다운로드 경로 지정
                          train = True, #True로 하면 훈련 데이터로 다운받게 된다,즉 FALSE이면 훈련데이터 다운 안받음
                           transform=transforms.ToTensor(), #텐서로 변환
                          download = True)

mnist_test = dsets.MNIST(root = 'MNIST_data/',
                        train = False,
                        transform=transforms.ToTensor(),
                        download = True)
#데이터로더를 사용하여 배치크기를 지정해준다. 만약 데이터셋과 데이터로더가 기억이 안나면 '미니배치와 데이터로드'복습

down_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size = batch_size,
                                          shuffle=True,
                                          drop_last = True) #drop_last는 마지막에 남는 데이터는 버림
#클래스 모델 설계 (conv + maxpool + fc)

class CNN(torch.nn.Module):
  
  def __init__(self):
    super(CNN,self).__init__()
    #첫번째 층
    #Img input shape(?,28,28,1)
    #CONV -> (?28,28,32)
    #POOL -> (?, 14,14,32)
    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1,32,kernel_size = 3, stride = 1, padding = 1), #kernel_size는 filter의 너비,높이사이즈 이다 3 * 3
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2)
    )

    #2번째 층
    #Img input shape(?,14,14,32)
    #Conv-> (?,14,14,64)
    #POOL -> (?,7,7,64)
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32,64, kernel_size = 3, stride = 1, padding = 1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2)
    )

    #FC층 7*7*64 -> 10output으로
    self.fc = torch.nn.Linear(7*7*64,10,bias = True)

    #FC층 한정으로 가중치 초기화
    torch.nn.init.xavier_uniform_(self.fc.weight)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0),-1)
        out = self.fc(out)
        return out
    
model = CNN().to(device)
linear = torch.nn.Linear(784,10,bias = True).to(device)

criterion = torch.nn.CrossEntropyLoss().to(device) #비용함수에 소프트맥스가 포함
optimizer = torch.optim.Adam(linear.parameters(), lr = learning_rate)

#총 배치 수 출력
total_batch = len(down_loader)
print('총 배치의 수 : {}'.format(total_batch))


for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in down_loader:
        X = X.view(-1,28*28).to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = linear(X)
        cost = criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9f}'.format(epoch + 1, avg_cost))


print('Learning Finished!')

[Epoch:    1] cost = 0.611851156
[Epoch:    2] cost = 0.344383001
[Epoch:    3] cost = 0.308516204
[Epoch:    4] cost = 0.291695327
[Epoch:    5] cost = 0.281251431
[Epoch:    6] cost = 0.274430633
